# Non-degnerate path probability

In some system having time-homogeneous transition rate matrix $R$ and initial probability distribution $p(x_{0},t_{0})$, the probability of a path $p(\mathcal{C}_{n})$ can be determined such that it includes the probability of all trajectories that follow that path:
\begin{equation}
p(\mathcal{C}_{n})=p(x_{0},t_{0}) \prod\limits_{i=0}\limits^{n-1} \frac{w(x_{i+1} \vert x_{i})}{w_{x_{i}}} \label{pcn}
\end{equation}
This formula uses the transition rates among states $w(x_{i+1} \vert x_{i})$ as given by the transition rate matrix and the total escape rate from a state:
\begin{equation}
w_{x_{i}} = \sum\limits_{\substack{j=0 \\ \neq i}}\limits^{n} w(x_{j} \vert x_{i})
\end{equation}
This path probability formula has a variety of uses. However, it lacks information about the completion time of the trajectories that follow that path. Conversely, a contracted path probability includes the probability of only those trajectories following that path that complete within some amount of time $\tau$. 

This contracted path probability is a consequence of the wait time distribution at each state. The wait time is the amount of time the system stays in one state before transitioning to the next state along the path. The distribution of wait times from a particular state $x_{i}$ is an exponential distribution with respect to the escape rate from that state:
\begin{equation}
\rho(\Delta t_{i} \vert x_{i}) = w_{x_{i}} e^{-\Delta t_{i} w_{x_{i}}}
\end{equation}
The path probaility is given by the convolution of each of the wait time distributions along the path times the path probability in the first equation:
\begin{align}
p(\mathcal{C}_{n},\tau) &=p(x_{0},t_{0})\prod\limits_{i=0}\limits^{n-1} \frac{w(x_{i+1} \vert x_{i})}{w_{x_{i}}} \rho(\Delta t_{0} \vert x_{0})*\rho(\Delta t_{1} \vert x_{1})*...*\rho(\Delta t_{n-1} \vert x_{n-1})*e^{-w_{x_{n}}(t-t_{n})} \\
&= p(x_{0},t_{0})\prod\limits_{i=0}\limits^{n-1} \frac{w(x_{i+1} \vert x_{i})}{w_{x_{i}}} \int_{t_{0}}^{t} dt_{1} w_{x_{0}} e^{-w_{x_{0}}(t_{1}-t_{0})} \times ... \times \int_{t_{n-1}}^{t} dt_{n} w_{x_{n-1}} e^{-w_{x_{n-1}}(t_{n}-t_{n-1})} \times e^{-w_{x_n}(t-t_{n})}
\end{align}
This contracted path probability has a general form that applies to all paths.  However, this formula can be simplified in two cases:

$\bullet$ fully degenerate path

$\bullet$ non-degenerate path

A path can be fully degenerate, partially degenerate, or non-degenerate based upon the escape rates of the states along the path. A path can be fully degenerate without all of its states being the same, as two states can have the same escape rate. By extension, a path must not include two instances of the same state or of two states with the same escape rate in order for it to be non-degenerate. The non-degenerate path probability is given by
\begin{equation}
p(\mathcal{C}_{n},\tau)=p(x_{0},t_{0}) \sum\limits_{i=0}\limits^{n-1} w(x_{i+1} \vert x_{i}) \sum\limits_{j=0}\limits^{n}  \frac{e^{-w_{x_{j}}(t-t_{0})}}{\prod\limits_{\substack{k=0 \\ \neq j}}\limits^{n} (w_{x_{k}}-w_{x_{j}})}
\end{equation}
The mean path occurrence time for a non-degenerate path is given by
\begin{equation}
\langle \tau \rangle_{\mathcal{C}_{n}} = \sum\limits_{i=0}\limits^{n} \frac{1}{w_{x_{i}}},
\end{equation}
and the variance of the path occurrence time is
\begin{equation}
\sigma^{2}_{\mathcal{C}_{n}} (\tau )= \sum\limits_{i=0}\limits^{n} \frac{1}{w_{x_{i}}^{2}}
\end{equation}

# Example

As an example, we consider a 4-state Markov model with transition rate matrix
\begin{equation}
R = 
\left(\begin{matrix}
-3 & 2 & 3 & 4 \\
1 & -6 & 3 & 4 \\
1 & 2 & -9 & 4 \\
1 & 2 & 3 & -12
\end{matrix}\right)
\end{equation}
There are 60 possible non-degenerate paths in this system as no two states can be repeated in a path in order for the path to be non-degenerate. Here we consider two similar paths: $\mathcal{C}_{2}=123$ and $\mathcal{C}_{2}=132$ as well as a longer path:$\mathcal{C}_{3}=1234$.

In [16]:
# Modules
import math

# Function to determine fully degenerate path probability
def det_prob(path,n,tau,wx,initialp,R):
    # Initial probability
    pr = initialp
    # Conditional probabilities
    for i in range(0,n):
        pr*=R[path[i]-1][path[i+1]-1]
    # Main sum
    summation = 0
    for i in range(0,n+1):
        temp = math.exp(-1*wx[path[i]-1]*tau)
        for j in range(0,n+1):
            if i != j:
                temp/=(wx[path[j]-1]-wx[path[i]-1])
        summation+=temp
    pr*=summation
    # Return result
    return pr
    
# Function to determine mean path occurrence time
def det_mean(wx,n,path):
    mn = 0;
    for i in range(0,n+1):
        mn+=(1/wx[path[i]-1])
    return mn
    
# Function to determine variance of path occurrence time
def det_var(wx,n,path):
    vr = 0;
    for i in range(0,n+1):
        temp = math.pow(wx[path[i]-1],2)
        vr+=(1/temp)
    return vr
    
# Wrapper function
def main():
    # Time scale of interest
    dt = 1
    # System
    pinit = [1,0,0,0]
    R = [[0 for i in range(4)] for j in range(4)]
    R[0][:] = [-3,1,1,1]
    R[1][:] = [2,-6,2,2]
    R[2][:] = [3,3,-9,3]
    R[3][:] = [4,4,4,-12]
    # Determine escape rates
    M = len(pinit)
    wx = [0]*(M)
    for i in range(0,M):
        wx[i] = -1*R[i][i]
    # Arrays for results
    probs = [0]*(3)
    means = [0]*(3)
    vari = [0]*(3)
    # Main determinations
    # Path 123
    path = [1,2,3]
    n = 2
    tau = (n-1)
    tau*=dt
    probs[0] = det_prob(path,n,tau,wx,pinit[0],R)
    means[0] = det_mean(wx,n,path)
    vari[0] = det_var(wx,n,path)
    # Printing
    print("Path")
    print(path)
    print("Path probability:")
    print(probs[0])
    print("Mean occurrence time:")
    print(means[0])
    print("Variance of occurrence time:")
    print(vari[0])
    # Path 132
    path = [1,3,2]
    n = 2
    tau = (n-1)
    tau*=dt
    probs[1] = det_prob(path,n,tau,wx,pinit[0],R)
    means[1] = det_mean(wx,n,path)
    vari[1] = det_var(wx,n,path)
    # Printing
    print("Path")
    print(path)
    print("Path probability:")
    print(probs[1])
    print("Mean occurrence time:")
    print(means[1])
    print("Variance of occurrence time:")
    print(vari[1])
    # Path 1234
    path = [1,2,3,4]
    n = 3
    tau = (n-1)
    tau*=dt
    probs[2] = det_prob(path,n,tau,wx,pinit[0],R)
    means[2] = det_mean(wx,n,path)
    vari[2] = det_var(wx,n,path)
    # Printing
    print("Path")
    print(path)
    print("Path probability:")
    print(probs[2])
    print("Mean occurrence time:")
    print(means[2])
    print("Variance of occurrence time:")
    print(vari[2])
        
main()

Path
[1, 2, 3]
Path probability:
0.004994774868735324
Mean occurrence time:
0.6111111111111112
Variance of occurrence time:
0.15123456790123457
Path
[1, 3, 2]
Path probability:
0.0074921623031029855
Mean occurrence time:
0.611111111111111
Variance of occurrence time:
0.15123456790123457
Path
[1, 2, 3, 4]
Path probability:
9.112463673312082e-05
Mean occurrence time:
0.6944444444444445
Variance of occurrence time:
0.15817901234567902


Despite having the same states visited, the first two paths have different probabilities because the transition rates along these paths are different. However, as the escape rates along these two paths are the same, they have the same mean and variance.

The longer path has a lower contracted path probability because the joint probability is spreading out over a larger number of paths. The mean occurrence time and variance are both larger than with the other two paths. This is expected as this path is longer and therefore will generally take longer to complete than a shorter path.